In [38]:
'''
Trying the configuration for keras tuner.
I create a dummy dataframe with categorical variables and 
categorical target.

'''
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from sklearn.model_selection import train_test_split
from pprint import pprint
from kerastuner.tuners import RandomSearch

In [2]:

def gen_dataset(dim):
  a1 = np.random.randint(0, 50, dim)
  a2 = a1 +  np.random.randint(0, 10, dim)
  b1=  np.random.randint(0, 50, dim)
  b2=  b1 * np.random.randint(0, 10, dim)    
  c1 = np.random.randint(0,100,dim)  
  data = pd.DataFrame(np.column_stack([a1,a2,b1,b2,c1]),columns=['a1','a2','b1','b2','c1'])
  return data

def to_categorical(df):
    for col in df.columns:
        if "a" in col:
            df[col] = df[col] % 10
        else:    
            df[col] = df[col] % 20
    return df    

def gen_dataset2(dim):
  a1 = np.random.randint(0, 50, dim)
  a2 = a1 * np.random.uniform(0.8,1.2,dim)
  b1=  np.random.randint(0, 50, dim)
  b2=  b1 * np.random.randint(0, 10, dim)    
  c1 = np.random.randint(0,100,dim)  
  data = pd.DataFrame(np.column_stack([a1,a2,b1,b2,c1]),columns=['a1','a2','b1','b2','c1'])
  return data

def addy(pdd_orig,ycalc):
  pdcopy = pdd_orig.copy()
  pdcopy["y"] = ycalc
  return pdcopy

In [3]:
#generate a random dataframe
pdd = gen_dataset(100)
test_set = addy(pdd, (pdd["a1"]*1+1*pdd["b1"])%2  ) 
print( test_set.head(10))

   a1  a2  b1   b2  c1  y
0  16  19  32  256  44  0
1  12  13  18  162  90  0
2  26  27  38   76  38  0
3  34  37  17  119  79  1
4   6  15  12  108  68  0
5   1   9  37    0   8  0
6   2   3  44  396  23  0
7   0   3  12    0  93  0
8   2   7  27  108  99  1
9   2   8  44  264  98  0


In [4]:
#convert this dataframe to categorical variables
df2 = to_categorical(test_set)
df2.head(10)

,a1,a2,b1,b2,c1,y
0,6,9,12,16,4,0
1,2,3,18,2,10,0
2,6,7,18,16,18,0
3,4,7,17,19,19,1
4,6,5,12,8,8,0
5,1,9,17,0,8,0
6,2,3,4,16,3,0
7,0,3,12,0,13,0
8,2,7,7,8,19,1
9,2,8,4,4,18,0


In [5]:
#create X and y variables
X = df2.drop('y', axis=1)
y = df2.y

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [7]:
X_train.head(10)

,a1,a2,b1,b2,c1
83,1,7,7,2,2
62,5,6,0,0,1
37,9,1,9,1,15
40,3,5,7,7,0
42,7,6,7,1,10
35,7,6,6,4,8
11,8,8,2,12,7
55,9,2,14,16,15
22,0,6,8,12,6
91,6,2,0,0,6


In [39]:
#I tried multiple configurations at once
def tune_optimizer_model(hp):
    model = keras.Sequential()
    
    #hyperparameters to test
    activation = hp.Choice('activation',['softmax', 'softplus',  'relu', 'tanh', 'sigmoid',  'linear'])
    optimizer = hp.Choice('optimizer', ['adam', 'sgd', 'rmsprop'])
    units_hp =hp.Int('units',min_value=8,max_value=128,step=16)
    model.add(keras.layers.Dense(units=units_hp, activation=activation, input_shape=[X_train.shape[1]]))
    drop_rate_hp = hp.Choice('drop_rate',[0.0,0.1,0.2,0.3,0.3,0.6,0.8])
        
    model.add(keras.layers.Dropout(rate=drop_rate_hp))    
    model.add(keras.layers.Dense(1, activation='sigmoid'))

    

    model.compile(
        optimizer=optimizer,
        loss = 'binary_crossentropy', 
        metrics = ['accuracy'])
    return model

In [41]:
tuner = RandomSearch(
    tune_optimizer_model,
    objective='val_accuracy',
    max_trials=10,
    executions_per_trial=2,
    directory='test_dir', 
    project_name='tune_optimizer9',
    seed=12
)

tuner.search_space_summary()

In [42]:
#call search method for really triggering the configuration
tuner.search(x=X_train,y=y_train,epochs=10,validation_data=(X_test, y_test))

Train on 80 samples, validate on 20 samples
Epoch 1/10
80/80 [==============================] - ETA: 0s - loss: 0.7779 - accuracy: 0.56 - 0s 6ms/sample - loss: 0.8463 - accuracy: 0.5625 - val_loss: 0.8768 - val_accuracy: 0.5500
Epoch 2/10
80/80 [==============================] - ETA: 0s - loss: 0.7669 - accuracy: 0.53 - 0s 497us/sample - loss: 0.8057 - accuracy: 0.5500 - val_loss: 0.8511 - val_accuracy: 0.5500
Epoch 3/10
80/80 [==============================] - ETA: 0s - loss: 0.6666 - accuracy: 0.62 - 0s 250us/sample - loss: 0.7554 - accuracy: 0.5750 - val_loss: 0.8270 - val_accuracy: 0.5500
Epoch 4/10
80/80 [==============================] - ETA: 0s - loss: 0.8065 - accuracy: 0.46 - 0s 189us/sample - loss: 0.7162 - accuracy: 0.5625 - val_loss: 0.8173 - val_accuracy: 0.5500
Epoch 5/10
80/80 [==============================] - ETA: 0s - loss: 0.6461 - accuracy: 0.56 - 0s 121us/sample - loss: 0.7227 - accuracy: 0.6000 - val_loss: 0.8058 - val_accuracy: 0.5500
Epoch 6/10
80/80 [==========

Train on 80 samples, validate on 20 samples
Epoch 1/10
80/80 [==============================] - ETA: 0s - loss: 3.8841 - accuracy: 0.50 - 1s 6ms/sample - loss: 4.3470 - accuracy: 0.5125 - val_loss: 2.5693 - val_accuracy: 0.4000
Epoch 2/10
80/80 [==============================] - ETA: 0s - loss: 3.9183 - accuracy: 0.50 - 0s 411us/sample - loss: 3.7281 - accuracy: 0.5375 - val_loss: 2.3016 - val_accuracy: 0.4000
Epoch 3/10
80/80 [==============================] - ETA: 0s - loss: 2.5235 - accuracy: 0.56 - 0s 838us/sample - loss: 3.2464 - accuracy: 0.5000 - val_loss: 2.0827 - val_accuracy: 0.4500
Epoch 4/10
80/80 [==============================] - ETA: 0s - loss: 5.3064 - accuracy: 0.46 - 0s 184us/sample - loss: 4.6051 - accuracy: 0.4750 - val_loss: 1.9369 - val_accuracy: 0.4500
Epoch 5/10
80/80 [==============================] - ETA: 0s - loss: 3.2388 - accuracy: 0.50 - 0s 742us/sample - loss: 3.0764 - accuracy: 0.4250 - val_loss: 1.7941 - val_accuracy: 0.5000
Epoch 6/10
80/80 [==========

Train on 80 samples, validate on 20 samples
Epoch 1/10
80/80 [==============================] - ETA: 0s - loss: 3.5719 - accuracy: 0.40 - 0s 6ms/sample - loss: 3.1639 - accuracy: 0.5000 - val_loss: 2.4375 - val_accuracy: 0.6000
Epoch 2/10
80/80 [==============================] - ETA: 0s - loss: 3.3576 - accuracy: 0.53 - 0s 230us/sample - loss: 2.8451 - accuracy: 0.5875 - val_loss: 2.1960 - val_accuracy: 0.5000
Epoch 3/10
80/80 [==============================] - ETA: 0s - loss: 4.9694 - accuracy: 0.40 - 0s 188us/sample - loss: 3.6072 - accuracy: 0.4875 - val_loss: 1.9842 - val_accuracy: 0.5000
Epoch 4/10
80/80 [==============================] - ETA: 0s - loss: 3.2839 - accuracy: 0.46 - 0s 258us/sample - loss: 3.5202 - accuracy: 0.4875 - val_loss: 1.7777 - val_accuracy: 0.5000
Epoch 5/10
80/80 [==============================] - ETA: 0s - loss: 3.7678 - accuracy: 0.43 - 0s 213us/sample - loss: 2.7197 - accuracy: 0.5625 - val_loss: 1.5803 - val_accuracy: 0.3500
Epoch 6/10
80/80 [==========

Train on 80 samples, validate on 20 samples
Epoch 1/10
80/80 [==============================] - ETA: 0s - loss: 4.0873 - accuracy: 0.46 - 0s 5ms/sample - loss: 3.1196 - accuracy: 0.4625 - val_loss: 1.2905 - val_accuracy: 0.5500
Epoch 2/10
80/80 [==============================] - ETA: 0s - loss: 1.8081 - accuracy: 0.46 - 0s 1ms/sample - loss: 1.9582 - accuracy: 0.4625 - val_loss: 1.3486 - val_accuracy: 0.6000
Epoch 3/10
80/80 [==============================] - ETA: 0s - loss: 2.7345 - accuracy: 0.31 - 0s 365us/sample - loss: 2.2034 - accuracy: 0.4125 - val_loss: 0.8666 - val_accuracy: 0.4500
Epoch 4/10
80/80 [==============================] - ETA: 0s - loss: 1.3811 - accuracy: 0.50 - 0s 144us/sample - loss: 1.2108 - accuracy: 0.5250 - val_loss: 1.0487 - val_accuracy: 0.5500
Epoch 5/10
80/80 [==============================] - ETA: 0s - loss: 1.0913 - accuracy: 0.53 - 0s 167us/sample - loss: 1.3121 - accuracy: 0.5250 - val_loss: 0.8577 - val_accuracy: 0.2500
Epoch 6/10
80/80 [============

Train on 80 samples, validate on 20 samples
Epoch 1/10
80/80 [==============================] - ETA: 0s - loss: 0.6833 - accuracy: 0.62 - 0s 5ms/sample - loss: 0.6910 - accuracy: 0.5500 - val_loss: 0.6791 - val_accuracy: 0.6000
Epoch 2/10
80/80 [==============================] - ETA: 0s - loss: 0.6812 - accuracy: 0.65 - 0s 304us/sample - loss: 0.6884 - accuracy: 0.5625 - val_loss: 0.6789 - val_accuracy: 0.5500
Epoch 3/10
80/80 [==============================] - ETA: 0s - loss: 0.6834 - accuracy: 0.53 - 0s 155us/sample - loss: 0.6873 - accuracy: 0.5750 - val_loss: 0.6783 - val_accuracy: 0.5500
Epoch 4/10
80/80 [==============================] - ETA: 0s - loss: 0.6913 - accuracy: 0.53 - 0s 195us/sample - loss: 0.6861 - accuracy: 0.6250 - val_loss: 0.6780 - val_accuracy: 0.6000
Epoch 5/10
80/80 [==============================] - ETA: 0s - loss: 0.6946 - accuracy: 0.50 - 0s 178us/sample - loss: 0.6852 - accuracy: 0.6250 - val_loss: 0.6775 - val_accuracy: 0.6000
Epoch 6/10
80/80 [==========

...

Train on 80 samples, validate on 20 samples
Epoch 1/10
80/80 [==============================] - ETA: 0s - loss: 1.0017 - accuracy: 0.68 - 0s 6ms/sample - loss: 1.0914 - accuracy: 0.6125 - val_loss: 0.7715 - val_accuracy: 0.4500
Epoch 2/10
80/80 [==============================] - ETA: 0s - loss: 0.6667 - accuracy: 0.62 - 0s 1ms/sample - loss: 0.7814 - accuracy: 0.6125 - val_loss: 0.6998 - val_accuracy: 0.5000
Epoch 3/10
80/80 [==============================] - ETA: 0s - loss: 0.8382 - accuracy: 0.71 - 0s 1ms/sample - loss: 0.8117 - accuracy: 0.5875 - val_loss: 0.6814 - val_accuracy: 0.5500
Epoch 4/10
80/80 [==============================] - ETA: 0s - loss: 0.8891 - accuracy: 0.53 - 0s 297us/sample - loss: 0.7749 - accuracy: 0.5750 - val_loss: 0.6649 - val_accuracy: 0.5500
Epoch 5/10
80/80 [==============================] - ETA: 0s - loss: 0.7749 - accuracy: 0.50 - 0s 594us/sample - loss: 0.8085 - accuracy: 0.5125 - val_loss: 0.6502 - val_accuracy: 0.6000
Epoch 6/10
80/80 [==============

Train on 80 samples, validate on 20 samples
Epoch 1/10
80/80 [==============================] - ETA: 0s - loss: 0.7576 - accuracy: 0.46 - 0s 5ms/sample - loss: 0.7199 - accuracy: 0.4750 - val_loss: 0.7563 - val_accuracy: 0.4000
Epoch 2/10
80/80 [==============================] - ETA: 0s - loss: 0.6922 - accuracy: 0.53 - 0s 421us/sample - loss: 0.6818 - accuracy: 0.5625 - val_loss: 0.7347 - val_accuracy: 0.4000
Epoch 3/10
80/80 [==============================] - ETA: 0s - loss: 0.7106 - accuracy: 0.53 - 0s 789us/sample - loss: 0.6684 - accuracy: 0.5750 - val_loss: 0.7268 - val_accuracy: 0.5000
Epoch 4/10
80/80 [==============================] - ETA: 0s - loss: 0.7198 - accuracy: 0.56 - 0s 1ms/sample - loss: 0.6622 - accuracy: 0.6000 - val_loss: 0.7212 - val_accuracy: 0.5500
Epoch 5/10
80/80 [==============================] - ETA: 0s - loss: 0.6890 - accuracy: 0.59 - 0s 935us/sample - loss: 0.6521 - accuracy: 0.6250 - val_loss: 0.7168 - val_accuracy: 0.6000
Epoch 6/10
80/80 [============

Train on 80 samples, validate on 20 samples
Epoch 1/10
80/80 [==============================] - ETA: 0s - loss: 0.9342 - accuracy: 0.59 - 0s 6ms/sample - loss: 0.8994 - accuracy: 0.5625 - val_loss: 0.6997 - val_accuracy: 0.7000
Epoch 2/10
80/80 [==============================] - ETA: 0s - loss: 0.8113 - accuracy: 0.53 - 0s 349us/sample - loss: 0.8764 - accuracy: 0.5000 - val_loss: 0.6736 - val_accuracy: 0.7000
Epoch 3/10
80/80 [==============================] - ETA: 0s - loss: 0.9307 - accuracy: 0.50 - 0s 235us/sample - loss: 0.8176 - accuracy: 0.5250 - val_loss: 0.6588 - val_accuracy: 0.7000
Epoch 4/10
80/80 [==============================] - ETA: 0s - loss: 0.8864 - accuracy: 0.46 - 0s 152us/sample - loss: 0.8218 - accuracy: 0.4875 - val_loss: 0.6600 - val_accuracy: 0.6000
Epoch 5/10
80/80 [==============================] - ETA: 0s - loss: 0.8875 - accuracy: 0.43 - 0s 129us/sample - loss: 0.7916 - accuracy: 0.5500 - val_loss: 0.6423 - val_accuracy: 0.6000
Epoch 6/10
80/80 [==========

Train on 80 samples, validate on 20 samples
Epoch 1/10
80/80 [==============================] - ETA: 0s - loss: 0.9028 - accuracy: 0.37 - 0s 5ms/sample - loss: 0.9146 - accuracy: 0.3750 - val_loss: 0.7411 - val_accuracy: 0.4500
Epoch 2/10
80/80 [==============================] - ETA: 0s - loss: 0.8481 - accuracy: 0.46 - 0s 371us/sample - loss: 0.8533 - accuracy: 0.4375 - val_loss: 0.6953 - val_accuracy: 0.4500
Epoch 3/10
80/80 [==============================] - ETA: 0s - loss: 0.7488 - accuracy: 0.43 - 0s 728us/sample - loss: 0.7675 - accuracy: 0.4000 - val_loss: 0.6628 - val_accuracy: 0.6500
Epoch 4/10
80/80 [==============================] - ETA: 0s - loss: 0.7952 - accuracy: 0.46 - 0s 786us/sample - loss: 0.7585 - accuracy: 0.4875 - val_loss: 0.6413 - val_accuracy: 0.8000
Epoch 5/10
80/80 [==============================] - ETA: 0s - loss: 0.7136 - accuracy: 0.50 - 0s 351us/sample - loss: 0.7359 - accuracy: 0.4125 - val_loss: 0.6300 - val_accuracy: 0.7000
Epoch 6/10
80/80 [==========

INFO:tensorflow:Oracle triggered exit


In [43]:
tuner.results_summary()

In [44]:
#dump best hyperparameters
tuner.get_best_hyperparameters()[0].values

{'activation': 'sigmoid', 'optimizer': 'adam', 'units': 72, 'drop_rate': 0.1}

In [45]:
#dump best model
tuner.get_best_models()[0].summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 72)                432       
_________________________________________________________________
dropout (Dropout)            (None, 72)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 73        
Total params: 505
Trainable params: 505
Non-trainable params: 0
_________________________________________________________________
